In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# For example, here's several helpful packages to load
!pip3 install --upgrade tensorflow
!pip3 install --upgrade tensorflow-probability
!pip install --upgrade "tensorflow-probability[tf]"



import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
# 設定 seaborn 主題，這會影響圖表的風格和顏色
sns.set_theme()  # Apply the default Seaborn theme
%matplotlib inline
# Libraries for statistical and scientific computing
import statsmodels.api as sm
from scipy import stats
from pytorch_lightning.callbacks import EarlyStopping, Callback, ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler, StandardScaler

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions
tfpl = tfp.layers

print('TF version:', tf.__version__)
print('TFP version:', tfp.__version__)

from keras.models import Sequential, Model
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, LSTM, TimeDistributed, ConvLSTM2D, Bidirectional, Input, Concatenate
from keras.losses import SparseCategoricalCrossentropy, BinaryCrossentropy
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

from sklearn.preprocessing import StandardScaler, OneHotEncoder

from pathlib import Path

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

/Users/apple/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


TF version: 2.16.1
TFP version: 0.24.0


In [18]:
# 使用pandas讀取CSV文件
df = pd.read_csv('https://raw.githubusercontent.com/xc90529/noninvasive_data_visualization/main/noninvasive_data.csv')

# To display the top 5 rows
df.head(5)
#CO-心輸出量
#CI - 心輸出指數
#SV - 舒張容積
#SVI - 舒張容積指數
#SVV - 舒張容積變異度
#SVR - 舒張容積阻力
#SVRI - 舒張容積指數阻力
#PR - 脈搏率
#SYS - 收縮壓
#DIA - 舒張壓
#MAP - 平均動脈壓

,si1m,CO,CI,SV,SVI,SVV,SVR,SVRI,PR,SYS,DIA,MAP,ID,time
0,N,4.0,2.4,43,26.0,10,1673.0,2745.0,93,116,74,91,#_1,120.000000
1,N,4.0,2.4,43,26.0,10,1683.0,2760.0,92,118,74,91,#_1,120.333333
2,N,3.6,2.2,39,24.0,17,1978.0,3245.0,92,121,80,97,#_1,120.666667
3,N,3.6,2.2,41,25.0,17,1868.0,3065.0,88,118,76,93,#_1,121.000000
4,N,4.2,2.6,42,26.0,10,1335.0,2190.0,100,100,66,78,#_1,121.333333


In [19]:
# Calculating the Shock Index as PR / SYS
df['PR/SYS'] = df['PR'] / df['SYS']

# Displaying the first few rows of the dataframe to confirm the new variable
df.head()

,si1m,CO,CI,SV,SVI,SVV,SVR,SVRI,PR,SYS,DIA,MAP,ID,time,PR/SYS
0,N,4.0,2.4,43,26.0,10,1673.0,2745.0,93,116,74,91,#_1,120.000000,0.801724
1,N,4.0,2.4,43,26.0,10,1683.0,2760.0,92,118,74,91,#_1,120.333333,0.779661
2,N,3.6,2.2,39,24.0,17,1978.0,3245.0,92,121,80,97,#_1,120.666667,0.760331
3,N,3.6,2.2,41,25.0,17,1868.0,3065.0,88,118,76,93,#_1,121.000000,0.745763
4,N,4.2,2.6,42,26.0,10,1335.0,2190.0,100,100,66,78,#_1,121.333333,1.000000


In [20]:
# Remove the 'ID' column and the 'si1m' column from your DataFrame
columns_to_check = df.drop(columns=['ID', 'si1m', ])

# Use the duplicated method to find duplicate rows
duplicate_rows = df[df.duplicated(subset=columns_to_check, keep=False)]

# Print the content of duplicate rows
print("Duplicate rows:")
print(duplicate_rows)
# 要排除的列名
columns_to_exclude = ['ID', 'si1m']

# 选择除了要排除的列以外的其他列
columns_to_check = df.columns[~df.columns.isin(columns_to_exclude)]

# 使用 selected_columns 作为子集来检查重复行，保留第一个重复行
duplicate_rows_df = df[df.duplicated(subset=columns_to_check, keep='first')]

# 打印重复行的数量
print("重复行的数量：", duplicate_rows_df.shape[0])

# 打印重复行的内容
print("重复行：")
print(duplicate_rows_df)
# 重置索引
df.reset_index(drop=True, inplace=True)

# 在原始数据框 df 中删除重复行
df = df.drop(duplicate_rows_df.index)

# 现在 df 中已经不包含重复行
print(df)

Duplicate rows:
      si1m   CO        CI  SV        SVI  SVV     SVR         SVRI   PR  SYS  \
17619    N  3.1  1.962025  49  31.012658   18  1558.0   986.075949   64   97   
17620    N  3.1  1.962025  49  31.012658   18  1558.0   986.075949   64   97   
17621    Y  2.9  1.835443  46  29.113924   17  1643.0  1039.873418   63   94   
17622    N  2.9  1.835443  46  29.113924   17  1643.0  1039.873418   63   94   
17623    Y  2.8  1.772152  45  28.481013   18  1878.0  1188.607595   62  101   
...    ...  ...       ...  ..        ...  ...     ...          ...  ...  ...   
35911    N  3.1  2.300000  27  20.000000   16  2038.0  2790.000000  116  101   
35912    Y  3.1  2.200000  27  20.000000   16  2045.0  2800.000000  112  101   
35913    N  3.1  2.200000  27  20.000000   16  2045.0  2800.000000  112  101   
35914    Y  2.9  2.100000  29  21.000000   13  2150.0  2945.000000  102  101   
35915    N  2.9  2.100000  29  21.000000   13  2150.0  2945.000000  102  101   

       DIA  MAP    ID  

In [23]:
# 四舍五入 df 中的所有数值列到小数点后第五位
rounded_df = df.round(5)

rounded_df.head()

,si1m,CO,CI,SV,SVI,SVV,SVR,SVRI,PR,SYS,DIA,MAP,ID,time,PR/SYS
0,N,4.0,2.4,43,26.0,10,1673.0,2745.0,93,116,74,91,#_1,120.00000,0.80172
1,N,4.0,2.4,43,26.0,10,1683.0,2760.0,92,118,74,91,#_1,120.33333,0.77966
2,N,3.6,2.2,39,24.0,17,1978.0,3245.0,92,121,80,97,#_1,120.66667,0.76033
3,N,3.6,2.2,41,25.0,17,1868.0,3065.0,88,118,76,93,#_1,121.00000,0.74576
4,N,4.2,2.6,42,26.0,10,1335.0,2190.0,100,100,66,78,#_1,121.33333,1.00000


In [24]:
# 将 'si1m' 列中的 'Y' 转换为 1，'N' 转换为 0
rounded_df['si1m'] = rounded_df['si1m'].apply(lambda x: 1 if x == 'Y' else 0)

# 计算除 'ID' 列外的所有列的 Pearson 相关系数
correlation_matrix = rounded_df.drop('ID', axis=1).corr()

# 打印与 'si1m' 相关的相关系数
print(correlation_matrix['si1m'])

##選擇PR,SYS,SVI,MAP來預測PR/SYS

si1m      1.000000
CO        0.033251
CI        0.034217
SV       -0.297894
SVI      -0.336060
SVV       0.090539
SVR      -0.122073
SVRI     -0.053453
PR        0.551291
SYS      -0.400070
DIA      -0.045034
MAP      -0.254025
time     -0.052093
PR/SYS    0.648060
Name: si1m, dtype: float64


In [25]:
# 選擇病人 #_9 的數據
id9 = rounded_df[rounded_df['ID'] == '#_9']
id9

,si1m,CO,CI,SV,SVI,SVV,SVR,SVRI,PR,SYS,DIA,MAP,ID,time,PR/SYS
6590,0,3.6,2.3,46,29.0,19,1444.44444,2260.86957,78,91,57,69,#_9,120.00000,0.85714
6591,0,3.8,2.4,42,27.0,35,1452.63158,2300.00000,90,94,60,73,#_9,120.33333,0.95745
6592,0,4.3,2.8,48,31.0,22,1227.90698,1885.71429,89,88,59,70,#_9,120.66667,1.01136
6593,0,4.4,2.9,47,30.0,20,1236.36364,1875.86207,95,92,59,72,#_9,121.00000,1.03261
6594,0,4.2,2.7,44,28.0,22,1238.09524,1925.92593,96,89,56,69,#_9,121.33333,1.07865
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8796,0,3.0,2.0,42,27.0,21,1840.00000,2760.00000,73,90,61,73,#_9,1393.00000,0.81111
8797,0,2.5,1.6,35,22.0,21,2016.00000,3150.00000,73,81,56,67,#_9,1393.33333,0.90123
8798,0,3.4,2.2,46,30.0,34,1694.11765,2618.18182,74,89,63,76,#_9,1394.66667,0.83146
8799,0,2.4,1.5,35,22.0,34,1766.66667,2826.66667,68,70,44,57,#_9,1395.00000,0.97143


相關係數分析

機器學習建模

In [9]:
class LossLoggingCallback(Callback):
    def __init__(self):
        super().__init__()
        self.val_losses = []  # To store validation losses
        self.train_losses = []  # To store training losses

    def on_validation_epoch_end(self, trainer, pl_module):
        val_loss = trainer.callback_metrics["val_loss"].item()
        self.val_losses.append(val_loss)
        print(f"Epoch {trainer.current_epoch}: val_loss={val_loss}")

    def on_train_epoch_end(self, trainer, pl_module, unused=None):  # unused parameter for compatibility
        # Assuming you have a metric named "train_loss" logged
        if "train_loss" in trainer.callback_metrics:
            train_loss = trainer.callback_metrics["train_loss"].item()
            self.train_losses.append(train_loss)
            print(f"Epoch {trainer.current_epoch}: train_loss={train_loss}")

填充id9 120~1395的缺失值，以0.33333為單位

In [26]:
#找出time 120~1395 從120開始以1為單位 所有的缺失值
# Ensure 'time' is sorted
data_sorted =id9.sort_values(by='time')

# Generate the full range of desired time values
full_time_range = pd.Series(range(120, 1396))

# Find missing time values
missing_times = full_time_range[~full_time_range.isin(data_sorted['time'])]

# Print the first 10 missing time values
missing_times.head(10).tolist()


[124, 125, 126, 127, 128, 133, 134, 151, 159, 182]

In [27]:
# 從120開始以1為單位 所有的缺失值用上一筆資料進行填充
# Correct the process of filling values considering fractional times properly
filled_values_corrected_proper = pd.DataFrame()

for missing_time in missing_times:
    # Find the closest previous time point, correctly handling fractional times
    closest_time = data_sorted[data_sorted['time'] < missing_time]['time'].max()
    # Copy the data from the closest time point
    row_to_copy = data_sorted[data_sorted['time'] == closest_time].copy()
    # Update the time to the missing time
    row_to_copy['time'] = missing_time
    # Append to the corrected filled values DataFrame
    filled_values_corrected_proper = pd.concat([filled_values_corrected_proper, row_to_copy], ignore_index=True)

# Concatenate the original sorted data with the properly filled values and sort by time
final_data_properly_filled = pd.concat([data_sorted, filled_values_corrected_proper], ignore_index=True).sort_values(by='time')

# To ensure accuracy, let's check the filled values, especially around the first few missing times, with accurate reference points
check_points = [120, 120.33333, 120.66667, 121]  # Reference points to check if the fill was correct
final_check = final_data_properly_filled[final_data_properly_filled['time'].isin(check_points + missing_times.tolist())]

final_check.head(10)  # Displaying more rows to observe the transition around the mentioned check points and first few filled values


,si1m,CO,CI,SV,SVI,SVV,SVR,SVRI,PR,SYS,DIA,MAP,ID,time,PR/SYS
0,0,3.6,2.3,46,29.0,19,1444.44444,2260.86957,78,91,57,69,#_9,120.00000,0.85714
1,0,3.8,2.4,42,27.0,35,1452.63158,2300.00000,90,94,60,73,#_9,120.33333,0.95745
2,0,4.3,2.8,48,31.0,22,1227.90698,1885.71429,89,88,59,70,#_9,120.66667,1.01136
3,0,4.4,2.9,47,30.0,20,1236.36364,1875.86207,95,92,59,72,#_9,121.00000,1.03261
2211,0,4.4,2.8,47,30.0,15,1236.36364,1942.85714,94,92,59,72,#_9,124.00000,1.02174
2212,0,4.4,2.8,47,30.0,15,1236.36364,1942.85714,94,92,59,72,#_9,125.00000,1.02174
2213,0,4.4,2.8,47,30.0,15,1236.36364,1942.85714,94,92,59,72,#_9,126.00000,1.02174
2214,0,4.4,2.8,47,30.0,15,1236.36364,1942.85714,94,92,59,72,#_9,127.00000,1.02174
2215,0,4.4,2.8,47,30.0,15,1236.36364,1942.85714,94,92,59,72,#_9,128.00000,1.02174
2216,0,5.0,3.2,56,36.0,12,992.00000,1550.00000,90,89,52,66,#_9,133.00000,1.01124


In [28]:
# Generate a list of the specific decimal time points from 120 to 1395, adding .3333333 to each
specific_decimal_times = [x + 0.33333 for x in range(120, 1396)]

# Check which of these specific times are missing in the final_data_properly_filled DataFrame
missing_specific_decimal_times = [time for time in specific_decimal_times if time not in final_data_properly_filled['time'].round(5).tolist()]

# Display the missing times
missing_specific_decimal_times[:10]


[123.33333,
 124.33333,
 125.33333,
 126.33333,
 127.33333,
 128.33333,
 133.33333,
 150.33333,
 151.33333,
 158.33333]

In [29]:
# Initialize a DataFrame to hold the filled values for missing specific decimal times
filled_values_for_missing_decimal = pd.DataFrame()

# Iterate over the sorted list of missing specific decimal times
for missing_time in missing_specific_decimal_times:
    # Find the closest previous time point
    closest_time = final_data_properly_filled[final_data_properly_filled['time'] < missing_time]['time'].max()
    # Copy the data from the closest time point
    row_to_copy = final_data_properly_filled[final_data_properly_filled['time'] == closest_time].copy()
    # Update the time to the missing time
    row_to_copy['time'] = missing_time
    # Append to the filled values DataFrame
    filled_values_for_missing_decimal = pd.concat([filled_values_for_missing_decimal, row_to_copy], ignore_index=True)

# Concatenate the original data with the filled values and sort by 'time'
final_data_with_filled_decimal = pd.concat([final_data_properly_filled, filled_values_for_missing_decimal], ignore_index=True).sort_values(by='time')

# Checking a sample of the filled values
final_data_with_filled_decimal[final_data_with_filled_decimal['time'].isin(missing_specific_decimal_times)].head()


,si1m,CO,CI,SV,SVI,SVV,SVR,SVRI,PR,SYS,DIA,MAP,ID,time,PR/SYS
2526,0,4.4,2.8,47,30.0,15,1236.36364,1942.85714,94,92,59,72,#_9,123.33333,1.02174
2527,0,4.4,2.8,47,30.0,15,1236.36364,1942.85714,94,92,59,72,#_9,124.33333,1.02174
2528,0,4.4,2.8,47,30.0,15,1236.36364,1942.85714,94,92,59,72,#_9,125.33333,1.02174
2529,0,4.4,2.8,47,30.0,15,1236.36364,1942.85714,94,92,59,72,#_9,126.33333,1.02174
2530,0,4.4,2.8,47,30.0,15,1236.36364,1942.85714,94,92,59,72,#_9,127.33333,1.02174


In [30]:
# Generate a list of the specific decimal time points from 120.66666666667 to 1395, adding .66666666667 to each integer
specific_decimal_times_66667 = [120.66667 + x for x in range(0, 1394 - 120 + 1)]

# Check which of these specific times are missing in the final_data_with_filled_decimal DataFrame
missing_specific_decimal_times_66667 = [time for time in specific_decimal_times_66667 if time not in final_data_with_filled_decimal['time'].round(11).tolist()]

# Display the missing times
missing_specific_decimal_times_66667[:10]


[123.66667,
 124.66667,
 125.66667,
 126.66667,
 127.66667,
 133.66667,
 150.66667,
 158.66667,
 176.66667,
 181.66667]

In [31]:
# Adjust the list of missing specific decimal times to exclude any time points beyond 1395
adjusted_missing_times_66667 = [time for time in missing_specific_decimal_times_66667 if time <= 1395]

# Initialize a new DataFrame for adjusted filling
adjusted_filled_values_for_missing_decimal_66667 = pd.DataFrame()

# Ensure the time column is rounded to match the precision of missing times for accurate comparison
final_data_with_filled_decimal['time'] = final_data_with_filled_decimal['time'].round(11)

# Iterate over the adjusted list of missing specific decimal times ending with .66667
for missing_time in adjusted_missing_times_66667:
    # Find the closest previous time point
    closest_time = final_data_with_filled_decimal[final_data_with_filled_decimal['time'] < missing_time]['time'].max()
    # Copy the data from the closest time point
    row_to_copy = final_data_with_filled_decimal[final_data_with_filled_decimal['time'] == closest_time].copy()
    # Update the time to the missing time
    row_to_copy['time'] = missing_time
    # Append to the filled values DataFrame
    adjusted_filled_values_for_missing_decimal_66667 = pd.concat([adjusted_filled_values_for_missing_decimal_66667, row_to_copy], ignore_index=True)

# Concatenate the current data with the adjusted filled values and sort by 'time'
final_data_adjusted_with_all_filled = pd.concat([final_data_with_filled_decimal, adjusted_filled_values_for_missing_decimal_66667], ignore_index=True).sort_values(by='time')

# Show a sample of the filled values to verify, focusing on the adjusted range
final_data_adjusted_with_all_filled[final_data_adjusted_with_all_filled['time'].isin(adjusted_missing_times_66667)].head()


,si1m,CO,CI,SV,SVI,SVV,SVR,SVRI,PR,SYS,DIA,MAP,ID,time,PR/SYS
3184,0,4.4,2.8,47,30.0,15,1236.36364,1942.85714,94,92,59,72,#_9,123.66667,1.02174
3185,0,4.4,2.8,47,30.0,15,1236.36364,1942.85714,94,92,59,72,#_9,124.66667,1.02174
3186,0,4.4,2.8,47,30.0,15,1236.36364,1942.85714,94,92,59,72,#_9,125.66667,1.02174
3187,0,4.4,2.8,47,30.0,15,1236.36364,1942.85714,94,92,59,72,#_9,126.66667,1.02174
3188,0,4.4,2.8,47,30.0,15,1236.36364,1942.85714,94,92,59,72,#_9,127.66667,1.02174


In [32]:
final_data_adjusted_with_all_filled

,si1m,CO,CI,SV,SVI,SVV,SVR,SVRI,PR,SYS,DIA,MAP,ID,time,PR/SYS
0,0,3.6,2.3,46,29.0,19,1444.44444,2260.86957,78,91,57,69,#_9,120.00000,0.85714
1,0,3.8,2.4,42,27.0,35,1452.63158,2300.00000,90,94,60,73,#_9,120.33333,0.95745
2,0,4.3,2.8,48,31.0,22,1227.90698,1885.71429,89,88,59,70,#_9,120.66667,1.01136
3,0,4.4,2.9,47,30.0,20,1236.36364,1875.86207,95,92,59,72,#_9,121.00000,1.03261
4,0,4.2,2.7,44,28.0,22,1238.09524,1925.92593,96,89,56,69,#_9,121.33333,1.07865
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3179,0,2.5,1.6,35,22.0,21,2016.00000,3150.00000,73,81,56,67,#_9,1394.00000,0.90123
3180,0,2.5,1.6,35,22.0,21,2016.00000,3150.00000,73,81,56,67,#_9,1394.33333,0.90123
3181,0,3.4,2.2,46,30.0,34,1694.11765,2618.18182,74,89,63,76,#_9,1394.66667,0.83146
3182,0,2.4,1.5,35,22.0,34,1766.66667,2826.66667,68,70,44,57,#_9,1395.00000,0.97143


In [33]:
# 進行特徵選取
selected_features_df = final_data_adjusted_with_all_filled[['si1m', 'SV', 'SVI', 'PR', 'SYS', 'MAP',  'PR/SYS',  'time']]
# 將'time'設置為索引
selected_features_df .set_index('time', inplace=True)
selected_features_df 

,si1m,SV,SVI,PR,SYS,MAP,PR/SYS
time,,,,,,,
120.00000,0,46,29.0,78,91,69,0.85714
120.33333,0,42,27.0,90,94,73,0.95745
120.66667,0,48,31.0,89,88,70,1.01136
121.00000,0,47,30.0,95,92,72,1.03261
121.33333,0,44,28.0,96,89,69,1.07865
...,...,...,...,...,...,...,...
1394.00000,0,35,22.0,73,81,67,0.90123
1394.33333,0,35,22.0,73,81,67,0.90123
1394.66667,0,46,30.0,74,89,76,0.83146


In [34]:
# 查看 si1m 變相的數量
si1m_counts = selected_features_df["si1m"].value_counts()

# 顯示 0 和 1 的數量
print(si1m_counts)

si1m
0    3610
1     217
Name: count, dtype: int64


In [26]:
# 將時間填充好且特徵選取過後的數據，另存到桌面上 CSV 文件
selected_features_df .to_csv('~/Desktop/id9_Fin_0402.csv')


In [35]:
# 参数化测试集占总数据的比例
test_ratio = 0.15

# 计算测试集大小
total_size = len(selected_features_df)
test_size = int(total_size * test_ratio)  # 计算测试集大小

# 分割索引
test_start = total_size - test_size

# 分割DataFrame为训练集和测试集
train_df = selected_features_df.iloc[:test_start]
test_df = selected_features_df.iloc[test_start:]

# 检查分割结果
print(f"训练集大小：{len(train_df)}")
print(f"测试集大小：{len(test_df)}")


训练集大小：3253
测试集大小：574


數據縮放後，將資料轉換為監督學習格式

In [36]:
# 使用MinMaxScaler进行缩放
scale = MinMaxScaler()
columns = train_df.columns

df_train_scaled = train_df.copy()
df_test_scaled = test_df.copy()
# 對訓練集的數值列进行擬合和转换，用相同的scaler避免数据泄露
df_train_scaled[columns] = scale.fit_transform(train_df[columns])
# 使用相同的scaler参数来转换测试集的数值列
df_test_scaled[columns] = scale.transform(test_df[columns])




In [41]:
#转换DataFrame为Numpy数组
data_train = np.array(train_df)
data_test = np.array(test_df)
#初始化空列表X和y
X_train = []
y_train = []
#定義歷史資料長度和預測長度
n_future = 1
n_timesteps = 30
n_features = 6 # number of sensors
n_outputs = 2 # binary states 0 and 1

In [42]:
for i in range(n_timesteps , len(data_train) - n_future + 1):
    X_train.append(data_train[i - n_timesteps : i, 1:])
    y_train.append(data_train[i : i+n_future, 0])
    
X_train, y_train = np.array(X_train), np.array(y_train)


In [43]:
print(X_train.shape)
print(y_train.shape)


(3223, 30, 6)
(3223, 1)


In [44]:
X_test = []
y_test = []
for i in range(n_timesteps , len(data_test) - n_future + 1):
    X_test.append(data_test[i - n_timesteps : i, 1:])
    y_test.append(data_test[i : i+n_future, 0])
X_test, y_test = np.array(X_test), np.array(y_test)

In [45]:
print('train data'.center(64, '-'))
print(X_train.shape)
print(y_train.shape)

print('test data'.center(64, '-'))
print(X_test.shape)
print(y_test.shape)

---------------------------train data---------------------------
(3223, 30, 6)
(3223, 1)
---------------------------test data----------------------------
(544, 30, 6)
(544, 1)


In [46]:
enc = OneHotEncoder()
y_train_oh = enc.fit_transform(y_train).toarray()
y_train_oh.shape

(3223, 2)

lstm模型建立


In [47]:
model = Sequential()
model.add(LSTM(256, input_shape=(n_timesteps, n_features)))
model.add(Dense(n_outputs, activation='softmax'))

model.summary() # reaches 97% and 86%



/Users/apple/Library/Python/3.9/lib/python/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 256)            │       269,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │           514 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 269,826 (1.03 MB)

 Trainable params: 269,826 (1.03 MB)

 Non-trainable params: 0 (0.00 B)

In [48]:
model = Sequential()
model.add(
    Bidirectional(
      LSTM(
          units=128,
          input_shape=[n_timesteps, n_features]
      )
    )
)
model.add(Dropout(rate=0.5))
model.add(Dense(units=128, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))

In [49]:
epochs = 50
batch_size = 64

callbacks = [
    ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=15, min_lr=0.0001
    ),
    EarlyStopping(monitor="val_loss", patience=15, verbose=1),
]
model.compile(
    optimizer="adam",
    loss=BinaryCrossentropy(),
    metrics=["accuracy"],
)
history = model.fit(
    X_train,
    y_train_oh,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_split=0.2,
    verbose=1,
)


Epoch 1/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - accuracy: 0.8900 - loss: 0.2793 - val_accuracy: 0.8744 - val_loss: 0.4685 - learning_rate: 0.0010
Epoch 2/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.9732 - loss: 0.1247 - val_accuracy: 0.8744 - val_loss: 0.4538 - learning_rate: 0.0010
Epoch 3/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - accuracy: 0.9722 - loss: 0.1254 - val_accuracy: 0.8744 - val_loss: 0.4325 - learning_rate: 0.0010
Epoch 4/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - accuracy: 0.9665 - loss: 0.1462 - val_accuracy: 0.8744 - val_loss: 0.4081 - learning_rate: 0.0010
Epoch 5/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.9736 - loss: 0.1208 - val_accuracy: 0.8744 - val_loss: 0.4534 - learning_rate: 0.0010
Epoch 6/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - accuracy: 0.9713 - loss: 0.1304 - val_accuracy: 0.8744 - val_loss: 0.4781 - learning_rate: 0.0010
Epoch 7/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - accuracy: 0.9674 - loss: 0.1390 - val_acc

In [50]:
y_pred_test = np.array(np.argmax(model.predict(X_test), axis=1))[...,np.newaxis]
y_pred_test.shape

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


(544, 1)

In [51]:
results_df = pd.DataFrame(y_pred_test, columns=['si1m'])
# 假设results_df是你的预测结果数据框，包含预测的状态
count_states = results_df['si1m'].value_counts()
print(count_states)


si1m
0    544
Name: count, dtype: int64


In [53]:
from sklearn.metrics import accuracy_score
train_accuracy = accuracy_score(y_test, y_pred_test)
print(f"Accuracy on train set: {train_accuracy * 100:.2f}%")

Accuracy on train set: 88.42%


In [54]:
y_pred_train = np.array(np.argmax(model.predict(X_train), axis=1))[...,np.newaxis]
y_pred_train.shape

101/101 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step


(3223, 1)

In [55]:
results_df = pd.DataFrame(y_pred_train, columns=['si1m'])
# 假设results_df是你的预测结果数据框，包含预测的状态
count_states_train = results_df['si1m'].value_counts()
print(count_states_train)

si1m
0    3223
Name: count, dtype: int64


In [56]:
from sklearn.metrics import accuracy_score

train_accuracy = accuracy_score(y_train, y_pred_train)
print(f"Accuracy on train set: {train_accuracy * 100:.2f}%")


Accuracy on train set: 95.22%
